SWE546 Final Project - Method 2

In this method, SVD is used.

Movielens 1M data set is used. The data set is divided into 2 part, one to be used for training, the other to be used for testing. 

Before performing SVD, a pre-processing is needed since SVD does not perform good with sparce matrices.
    1- rating averages per movies are obtained from the training data
    2- these ratings are subtracted from the rating matrix, including the zero values. 
    3- user offsets are found to see the per user bias


After SVD is performed, the rating matrix is reconstructed with lower rank. After that, in order to find the predicted ratings, rating average per movies and user offsets are added onto the reconstructed matrix.   

After the predicted ratings are obtained, the test data is used to calculate accuracy, precision, recall, TPR and FPR. These metrics are calculated by selecting a threshold of 3 as rating and the expectation was if the rating in the test data is >= than the threshold, then the prediction must also be >= than the threshold and the other way around. Any prediction that does not follow this metric is counted as either false positive or false negative. 

The performance metrics can be found as printed at end of this notebook.

In [1]:
import numpy as np
import pandas as pd
import time


rnames = ['user_id', 'movie_id', 'rating', 'timestamp'] 
mnames = ['MovieID', 'Title', 'Genres']
train = pd.read_table('../data/ml-1m/ra.train', sep='::',engine='python', header=None, names=rnames)
test = pd.read_table('../data/ml-1m/ra.test', sep='::',engine='python', header=None, names=rnames)

In [2]:
Ys = np.mat(train)
Ys

matrix([[        1,       595,         5, 978824268],
        [        1,       938,         4, 978301752],
        [        1,      2398,         4, 978302281],
        ..., 
        [     6040,       562,         5, 956704746],
        [     6040,      1096,         4, 956715648],
        [     6040,      1097,         4, 956715569]])

In [3]:
Yt = np.mat(test)
Yt

matrix([[        1,      1193,         5, 978300760],
        [        1,       661,         3, 978302109],
        [        1,       914,         3, 978301968],
        ..., 
        [     6040,      3017,         1, 956716519],
        [     6040,      2070,         4, 956715676],
        [     6040,      1419,         3, 956704056]])

In [31]:
Nr = 6040 # Number of users
Nc = 3952 # Number of movies

#Obtain Y matrix
Y = np.mat(np.zeros((Nr, Nc)))
Ysize = Y.shape[0]

MovieSum = np.mat(np.zeros((1,Nc)))
MovieCnt = np.mat(np.zeros((1,Nc)))

UserSum = np.mat(np.zeros((1,Nr)))
UserCnt = np.mat(np.zeros((1,Nr)))

for i in range(Ysize):
    user = Ys[i,0]
    movie = Ys[i,1]
    rating = Ys[i,2]
    
    MovieSum[0, movie-1] += rating
    MovieCnt[0, movie-1] += 1
    UserSum[0, user-1] += rating
    UserCnt[0, user-1] += 1
    Y[user-1, movie-1] = Ys[i, 2]
Y

matrix([[ 5.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [32]:
np.sum(Y)

21690.0

In [33]:
#get rating averages per movie
for i in range(Nc):
    if(MovieSum[0,i] != 0):
        MovieSum[0, i] = MovieSum[0,i] / MovieCnt[0,i]

#get rating averages per user
for i in range(Nr):
    if(UserSum[0,i] != 0):
        UserSum[0, i] = UserSum[0,i] / UserCnt[0,i]


In [34]:
MovieSum

matrix([[ 4.5,  2.8,  2. , ...,  0. ,  0. ,  4.5]])

In [35]:
UserSum

matrix([[ 4.20930233,  3.70588235,  4.02439024, ...,  0.        ,
          0.        ,  0.        ]])

In [95]:
#subtract the movie means from all ratings - including the 0 ones
Ysub = np.mat(np.zeros((Nr, Nc)))

for i in range(Nr):
    for j in range(Nc):
        Ysub[i, j] = Y[i, j] - MovieSum[0, j]


In [96]:
Ysub

matrix([[ 0.5, -2.8, -2. , ...,  0. ,  0. , -4.5],
        [-4.5, -2.8, -2. , ...,  0. ,  0. , -4.5],
        [-4.5, -2.8, -2. , ...,  0. ,  0. , -4.5],
        ..., 
        [-4.5, -2.8, -2. , ...,  0. ,  0. , -4.5],
        [-4.5, -2.8, -2. , ...,  0. ,  0. , -4.5],
        [-4.5, -2.8, -2. , ...,  0. ,  0. , -4.5]])

In [97]:
#find out the offset per user
AverageUserOff = np.mat(np.zeros((1, Nr)))

UserCnt = np.mat(np.zeros((1,Nr)))

#find out the user offsets per rating 
UserOff = np.mat(np.zeros((Nr, Nc)))

for i in range(Nr):
    for j in range(Nc):
        if(Y[i,j] != 0):
            UserOff[i, j] = Y[i,j] - MovieSum[0, j]
            AverageUserOff[0, i] += UserOff[i,j]
            UserCnt[0,i] += 1


In [98]:
for i in range(Nr):
    if(UserCnt[0, i] != 0):
        AverageUserOff[0, i] = AverageUserOff[0, i] / UserCnt[0,i]

AverageUserOff

matrix([[ 0.20711   ,  0.05948643,  0.15327994, ...,  0.        ,
          0.        ,  0.        ]])

In [99]:
Ysub

matrix([[ 0.5, -2.8, -2. , ...,  0. ,  0. , -4.5],
        [-4.5, -2.8, -2. , ...,  0. ,  0. , -4.5],
        [-4.5, -2.8, -2. , ...,  0. ,  0. , -4.5],
        ..., 
        [-4.5, -2.8, -2. , ...,  0. ,  0. , -4.5],
        [-4.5, -2.8, -2. , ...,  0. ,  0. , -4.5],
        [-4.5, -2.8, -2. , ...,  0. ,  0. , -4.5]])

In [100]:
#perform SVD on pre-processed matrix (Ysub) 
import scipy as sc
import scipy.linalg as la
U,S,Vt = la.svd(Ysub)

print('U')
print(U)

print('S')
print(S)

print('V')
print(Vt)

U
[[  1.24351900e-02   5.21865511e-02   4.19191584e-02 ...,  -1.69416707e-17
   -1.27943418e-18  -1.51498132e-17]
 [  1.18748143e-02   1.14401164e-01  -2.74249016e-02 ...,   8.90373740e-19
   -1.96684417e-17  -1.82428395e-17]
 [  1.24830280e-02   7.87131673e-02   1.28619246e-02 ...,  -1.55182713e-18
    1.64891878e-19   2.86965674e-18]
 ..., 
 [  1.28746417e-02  -7.12430494e-04  -4.43454684e-05 ...,   5.00000000e-01
    6.50899661e-18   5.31421346e-18]
 [  1.28746417e-02  -7.12430494e-04  -4.43454684e-05 ...,  -1.49302421e-19
    5.00000000e-01  -5.00000000e-01]
 [  1.28746417e-02  -7.12430494e-04  -4.43454684e-05 ...,  -1.49302421e-19
   -5.00000000e-01   5.00000000e-01]]
S
[  1.13396660e+04   1.05684176e+02   7.19282148e+01 ...,   1.02145720e-12
   5.66493605e-13   3.56294424e-14]
V
[[-0.0307778  -0.01917634 -0.01370276 ...,  0.          0.         -0.03083178]
 [ 0.07182302  0.01830801 -0.01047636 ...,  0.          0.         -0.00142611]
 [ 0.06214707  0.04497073  0.01128396 ...,  

In [101]:
print(S[:50])

[  1.13396660e+04   1.05684176e+02   7.19282148e+01   6.60142566e+01
   6.12285898e+01   5.97271029e+01   5.70542710e+01   5.40480069e+01
   5.06732250e+01   5.00552848e+01   4.81471485e+01   4.66632725e+01
   4.43178623e+01   4.25574860e+01   4.18856887e+01   4.15324785e+01
   4.08706529e+01   3.98891732e+01   3.81567492e+01   3.71220410e+01
   3.60677773e+01   3.43327554e+01   3.41174430e+01   3.34734460e+01
   3.21804733e+01   3.16432107e+01   3.04939640e+01   2.95961364e+01
   2.89582965e+01   2.78722091e+01   2.66564113e+01   2.53771090e+01
   2.49291202e+01   2.40365711e+01   2.37575101e+01   2.25726943e+01
   2.16215575e+01   1.77946473e+01   1.70448054e+01   1.64351235e+01
   1.49739928e+01   1.42029570e+01   1.34346380e+01   1.31609921e+01
   1.27690670e+01   1.16539899e+01   1.08741434e+01   9.41348119e+00
   7.13502498e-11   5.92206768e-11]


In [102]:
#select the rank from the meaning values of S matrix and obtain the diagonal matrix
rnk = 2

Sr = np.zeros_like(S)

Sr[0:rnk] = S[0:rnk]
Sig = la.diagsvd(Sr, Nr, Nc)

print('Sr')
print(Sr)

print('Sig')
print(Sig)

Sr
[ 11339.66596317    105.68417582      0.         ...,      0.              0.
      0.        ]
Sig
[[ 11339.66596317      0.              0.         ...,      0.              0.
       0.        ]
 [     0.            105.68417582      0.         ...,      0.              0.
       0.        ]
 [     0.              0.              0.         ...,      0.              0.
       0.        ]
 ..., 
 [     0.              0.              0.         ...,      0.              0.
       0.        ]
 [     0.              0.              0.         ...,      0.              0.
       0.        ]
 [     0.              0.              0.         ...,      0.              0.
       0.        ]]


In [103]:
#reconstruct the ratings matrix with lower rank
Yr = np.dot(np.dot(U, Sig), Vt)
Yr

array([[-3.94388057, -2.60309927, -1.99001917, ...,  0.        ,
         0.        , -4.35548308],
       [-3.27606028, -2.3608667 , -1.97182842, ...,  0.        ,
         0.        , -4.16894015],
       [-3.75922471, -2.56217631, -2.02682234, ...,  0.        ,
         0.        , -4.37620632],
       ..., 
       [-4.49878628, -2.80101197, -1.99973429, ...,  0.        ,
         0.        , -4.50115229],
       [-4.49878628, -2.80101197, -1.99973429, ...,  0.        ,
         0.        , -4.50115229],
       [-4.49878628, -2.80101197, -1.99973429, ...,  0.        ,
         0.        , -4.50115229]])

In [104]:
#Obtain the prediction matrix using the reconstructed matrix, average ratings per movie and user offsets
Yp = np.mat(np.zeros((Nr, Nc)))

for i in range(Nr):
    for j in range(Nc):
        Yp[i,j] = Yr[i,j] +  MovieSum[0, j] + AverageUserOff[0, i]

In [106]:
print(Yp)

[[  7.63229430e-01   4.04010731e-01   2.17090825e-01 ...,   2.07109996e-01
    2.07109996e-01   3.51626911e-01]
 [  1.28342615e+00   4.98619734e-01   8.76580153e-02 ...,   5.94864307e-02
    5.94864307e-02   3.90546282e-01]
 [  8.94055238e-01   3.91103632e-01   1.26457609e-01 ...,   1.53279944e-01
    1.53279944e-01   2.77073627e-01]
 ..., 
 [  1.21371692e-03  -1.01196855e-03   2.65713666e-04 ...,   0.00000000e+00
    0.00000000e+00  -1.15229047e-03]
 [  1.21371692e-03  -1.01196855e-03   2.65713666e-04 ...,   0.00000000e+00
    0.00000000e+00  -1.15229047e-03]
 [  1.21371692e-03  -1.01196855e-03   2.65713666e-04 ...,   0.00000000e+00
    0.00000000e+00  -1.15229047e-03]]


In [107]:
testSize = Yt.shape[0]

np.random.shuffle(Yt)

metric_threshold = 3
prediction_threshold = 0


TP = 0;
FP = 0; 
TN = 0;
FN = 0;

for t in range(testSize):
    user = Yt[t, 0] - 1
    movie = Yt[t, 1] - 1
    
    rating = Yt[t, 2]
    
    predict = Yp[user,movie]
    
    if (rating >= metric_threshold and predict >= prediction_threshold):
        TP += 1
    elif (rating < metric_threshold and predict < prediction_threshold):
        TN += 1
    elif (rating > metric_threshold and predict < prediction_threshold):
        FN += 1
    elif (rating < metric_threshold and predict > prediction_threshold):
        FP += 1

        
if ((TP + FN) == 0 ):
    TPR = 0
else:
    TPR = float(TP / float(TP+FN))
if ((FP + TN) == 0 ):
    FPR = 0
else:
    FPR = float(FP / float(FP+TN))
        
print('truePos = ' + repr(TP) + ', trueNeg = ' + repr(TN) + 
      ', falsePos = ' + repr(FP) + ', falseNeg = ' + repr(FN) + '\n')

print('True Positive Rate = ' + repr(TPR) + ', False Positive Rate = ' + repr(FPR) + '\n')

accuracy = float((TP + TN) / float(TP + FP + TN + FN + 1)) * 100
precision = float(TP / float(TP + FP + 1)) * 100
recall = float(TP / float(TP + FN + 1)) * 100

print('accuracy = ' + repr(accuracy) + '%, precision = ' + repr(precision) + 
      '%, recall = ' + repr(recall) + '%\n')


truePos = 25740, trueNeg = 3273, falsePos = 2271, falseNeg = 20123

True Positive Rate = 0.5612367267732159, False Positive Rate = 0.4096320346320346

accuracy = 56.43674136321195%, precision = 91.88919034699414%, recall = 56.12244897959183%

